In [1]:
import ujson as json
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plotly.plotly as py
import datetime
from __future__ import division

from moztelemetry import get_pings, get_pings_properties, get_one_ping_per_client

%pylab inline

Unable to parse whitelist (/home/hadoop/anaconda2/lib/python2.7/site-packages/moztelemetry/histogram-whitelists.json). Assuming all histograms are acceptable.
Populating the interactive namespace from numpy and matplotlib


In [36]:
# We grab pings starting from 2 days ago until [TimeWindow] days ago.
TimeWindow = 7

# Additional filters.
Filters = {
    'app': 'Firefox',
    
    # We sample 0.5% of pings. For testing, it is beter to use a small number
    # here (like 0.001) to speed up processing time.
    'fraction': 1,
    
    # Optionally restrict pings to a single channel.
     'channel': 'release',
}

# In case you need them!
VendorIDs = {
    'Intel': '0x8086',
    'NVIDIA': '0x10de',
    'AMD': '0x1002',
    'ATI': '0x1002'
}

In [37]:
###############################
# This section gathers pings. #
###############################

def run_get_pings():
    def fmt_date(d):
        return d.strftime("%Y%m%d")
    t1 = fmt_date(datetime.datetime.now() - datetime.timedelta(TimeWindow + 2)) # go back 16 days
    t2 = fmt_date(datetime.datetime.now() - datetime.timedelta(2)) # go back 2 days
    return get_pings(sc, build_id=(t1, t2), **Filters)

# Get pings for the parameters in the previous step.
raw_pings = run_get_pings()

In [38]:
######################################################################
# This section takes the raw ping list, then formats and filters it. #
######################################################################

# Map the pings into a more readable dictionary-like form. To see
# what these look like, execute "pings.take(1)".
unique_pings = get_pings_properties(raw_pings, [
  "clientId",
  "environment/build/version",
  "environment/system/os/name",
  "environment/system/os/version",
  "environment/system/gfx/adapters",
  "environment/system/gfx/monitors",
])
unique_pings = get_one_ping_per_client(unique_pings)

In [39]:
unique_pings.take(1)

[{'clientId': u'1e9d72d0-8271-4cbc-8c7a-44b5b98562fd',
  'environment/build/version': u'47.0',
  'environment/system/gfx/adapters': [{u'GPUActive': True,
    u'RAM': 384,
    u'description': u'ATI Radeon HD 4200',
    u'deviceID': u'0x9710',
    u'driver': u'aticfx64 aticfx64 aticfx32 aticfx32 atiumd64 atidxx64 atiumdag atidxx32 atiumdva atiumd6a atitmm64',
    u'driverDate': u'4-29-2013',
    u'driverVersion': u'8.970.100.1100',
    u'subsysID': u'3047103c',
    u'vendorID': u'0x1002'}],
  'environment/system/gfx/monitors': [{u'pseudoDisplay': False,
    u'refreshRate': 60,
    u'screenHeight': 720,
    u'screenWidth': 1280}],
  'environment/system/os/name': u'Windows_NT',
  'environment/system/os/version': u'6.1'}]

In [40]:
# We add two extra steps. The first rewrites the ping to have some
# information more easily accessible (like the primary adapter),
# and the second step removes any pings that don't have adapter
# information.
def rewrite_ping(p):
    adapters = p.get('environment/system/gfx/adapters', None)
    if not adapters:
        return None
    adapter = adapters[0]
            
    p['adapter'] = adapter

            
    # Convert the version to a tuple of integers.
    if 'driverVersion' in adapter:
        p['driverVersion'] = [int(n) for n in adapter['driverVersion'].split('.') if n.isdigit()]
    return p


pings = unique_pings#.map(rewrite_ping)
pings = pings.cache()

#pings = pings.cache()

In [41]:
# Observe the format of a random ping. This may take some time since it has to
# execute the pipeline.
#pings.take(1)
#pings.take(1)
#new_pings = pings.map(rewrite_ping).filter(filter_ping)
pings.take(1)

[{'clientId': u'1e9d72d0-8271-4cbc-8c7a-44b5b98562fd',
  'environment/build/version': u'47.0',
  'environment/system/gfx/adapters': [{u'GPUActive': True,
    u'RAM': 384,
    u'description': u'ATI Radeon HD 4200',
    u'deviceID': u'0x9710',
    u'driver': u'aticfx64 aticfx64 aticfx32 aticfx32 atiumd64 atidxx64 atiumdag atidxx32 atiumdva atiumd6a atitmm64',
    u'driverDate': u'4-29-2013',
    u'driverVersion': u'8.970.100.1100',
    u'subsysID': u'3047103c',
    u'vendorID': u'0x1002'}],
  'environment/system/gfx/monitors': [{u'pseudoDisplay': False,
    u'refreshRate': 60,
    u'screenHeight': 720,
    u'screenWidth': 1280}],
  'environment/system/os/name': u'Windows_NT',
  'environment/system/os/version': u'6.1'}]

In [42]:
# Count the total number of sessions in the dataset.
TotalSessions = pings.count()
print('Number of sessions: {0}'.format(TotalSessions))

Number of sessions: 191515


In [43]:
import json
import urllib2

gpu_db = json.load(urllib2.urlopen('https://raw.githubusercontent.com/jrmuizel/gpu-db/master/intel.json'))
devices = {}
for gen in gpu_db['8086'].items():
    for chipset in gen[1].items():
        for dev in chipset[1]:
            #print dev, gen[0]
            devices[int(dev,16)] = gen[0]

In [44]:
# Sample filter #1 - how many people are using Intel devices
# with a driver less than 8.15.10.2622? (bug 1175366).
def sample_filter_1(p):
    if p['environment/system/os/name'] != "Darwin":
        return False
    return True

interesting_pings = pings.filter(sample_filter_1)
interesting_pings.take(1)


[{'clientId': u'fdbec568-e91b-2348-bc1c-732ea66bd7d7',
  'environment/build/version': u'47.0',
  'environment/system/gfx/adapters': [{u'GPUActive': True,
    u'deviceID': u'0x0166',
    u'vendorID': u'0x8086'}],
  'environment/system/gfx/monitors': [{u'scale': 1,
    u'screenHeight': 800,
    u'screenWidth': 1280}],
  'environment/system/os/name': u'Darwin',
  'environment/system/os/version': u'13.4.0'}]

In [45]:
monitors = interesting_pings.map(lambda p: (len(p['environment/system/gfx/monitors']),))
monitors.countByKey()

dual_pings = interesting_pings.filter(lambda p: len(p['environment/system/gfx/monitors']) > 1 )
dual_pings.take(3)

scales = interesting_pings.map(lambda p: (len(set([o['scale'] for o in p['environment/system/gfx/monitors']])),))
multiple_scales = interesting_pings.count() -  scales.countByKey()[1]
print('{0} out of {1} sessions matched. ({2:.2f}%)'.format(
        multiple_scales,
        interesting_pings.count(),
       ((multiple_scales / interesting_pings.count()) * 100)))



383 out of 11569 sessions matched. (3.31%)


In [102]:




def webgl_fail(p):
    if p['payload/histograms/CANVAS_WEBGL_SUCCESS'][0] > 0:
        return True

     #   return True
    return False
fail_pings = interesting_pings.filter(webgl_fail)

def vga_fail(p):
    if 'vga' in p['adapter']['driver']:
        return True


     #   return True
    return False
vga_pings = fail_pings.filter(vga_fail)

def not_vga_fail(p):
    if 'vga' in p['adapter']['driver']:
        return False
     #   return True
    return True
not_vga_pings = fail_pings.filter(not_vga_fail)


amd1_pings = fail_pings.filter(lambda p: p['features']['webgl']['failureId'] == 'FEATURE_FAILURE_AMD1')



#TDRResults = sample_result_1.map(lambda p: p[DeviceResetReasonKey]).reduce(lambda x, y: x + y)
#help(pd.Series(TDRResults.values, DeviceResetReason))
#print(pd.Series(TDRResults.values[1:8],DeviceResetReason))
#pd.Series(TDRResults.values[1:8],DeviceResetReason).plot.bar()
#reduce(lambda x, y: sample_result_1.take(2))
#sample_result_1.take(1)[0]['payload/histograms/DEVICE_RESET_REASON'])
#fail_pings.take(1)
def match_count(a, b):
    print('{0} out of {1} sessions matched. ({2:.2f}%)'.format(
        a.count(),
        b.count(),
       ((a.count() / b.count()) * 100)))



def map_x_to_count(data, sourceKey):
    def extract(p):
        return (p['adapter'][sourceKey],)
    return data.map(extract).countByKey()    
match_count(interesting_pings, pings)    
match_count(fail_pings, interesting_pings)
match_count(vga_pings, fail_pings)
match_count(not_vga_pings, fail_pings)
match_count(amd1_pings, fail_pings)
#map_x_to_count(not_vga_pings, 'driverVersion')
#not_vga_pings.take(5)

#fail_pings.map(lambda p: (p['features']['webgl']['failureId'],)).countByKey()
#amd1_pings.take(23)
amd1_pings.map(lambda p: (p['adapter']['driverVersion'],)).countByKey()

4007 out of 17274 sessions matched. (23.20%)
103 out of 4007 sessions matched. (2.57%)
13 out of 103 sessions matched. (12.62%)
90 out of 103 sessions matched. (87.38%)
21 out of 103 sessions matched. (20.39%)


defaultdict(int,
            {u'6.1.7600.16385': 3,
             u'8.33.0.0': 1,
             u'8.420.0.0': 1,
             u'8.471.0.0': 1,
             u'8.479.0.0': 1,
             u'8.522.0.0': 1,
             u'8.56.1.15': 3,
             u'8.56.1.16': 5,
             u'8.593.100.0': 5})